In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [3]:
data = pd.read_csv('sensor_raw2.csv') #Cleaning Dataset
window_size = 14

In [4]:
def train_and_predict(data):
    X = []
    y = []

    for _, task_data in data.groupby('Class'):
        for i in range(0, len(task_data) - window_size):
            window = task_data.iloc[i:i + window_size]
            feature_vector = [window['AccX'].mean(),
                              window['AccY'].mean(),
                              window['AccZ'].mean(),
                              window['GyroX'].mean(),
                              window['GyroY'].mean(),
                              window['GyroZ'].mean()]
            X.append(feature_vector)
            y.append(window['Class'].values[0])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"\nModel Accuracy: {accuracy}")

    return model

def predict_classes(model, data):
    class_counts = {1: 0, 2: 0, 3: 0, 4: 0}

    for i in range(0, len(data) - window_size):
        window = data.iloc[i:i + window_size]
        feature_vector = [window['AccX'].mean(),
                          window['AccY'].mean(),
                          window['AccZ'].mean(),
                          window['GyroX'].mean(),
                          window['GyroY'].mean(),
                          window['GyroZ'].mean()]
        predicted_class = int(model.predict([feature_vector])[0])
        class_counts[predicted_class] += 1

    print("\nClass Counts at the End of the Journey:")
    for class_label, count in class_counts.items():
        print(f"Class {class_label}: {count} times")

# Train the model on all drivers combined
trained_model = train_and_predict(data)

# Predict classes at the end of the journey
predict_classes(trained_model, data)


Model Accuracy: 0.9622641509433962

Class Counts at the End of the Journey:
Class 1: 265 times
Class 2: 279 times
Class 3: 336 times
Class 4: 220 times
